<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_Valuation_Gap_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

end_date = '01-Apr-2025'
start_date = '01-Apr-2017'

Run date time (IST): 2024-06-16 15:40:26


In [2]:
!pip install yfinance

In [3]:
import pandas as pd
import numpy as np

mon = 'Dec'

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/'

mypps = pd.read_csv(path_mypf + 'myProspectsScrips.csv')
#mypps = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myProspectsScrips.csv')

mypfs = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myPortfolioStocks.csv')
mypfs = mypfs[mypfs['InPortfolio'].isin(['SV','DM'])]['Symbol']
mypfs = mypps[mypps['Symbol'].isin(mypfs)]['Stock']

#mypf = mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]
mypps = mypps[mypps['Criteria'].isin(['V40','V40N','VR','AR', 'V200'])]

stock_n100 = mypps['Symbol'].unique()

exclude = ['ACC','AMBUJACEM','CADILAHC','PGHH','SIEMENS', 'GILLETTE', 'NESTLEIND', 'SANOFI', 'AMARAJABAT', 'SCHAEFFLER ']

stock_n100 = mypps[~mypps['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100[:]
#stock_ids = ['HDFCBANK', 'COLPAL']
print('Stocks to be scanned:',len(stock_ids))

Stocks to be scanned: 134


In [4]:
df_latest_analysis = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myProspects-ATH-TTM.csv')
df_latest_analysis.columns

Index(['Type', 'Mar 2018', 'Mar 2019', 'Mar 2020', 'Mar 2021', 'Mar 2022',
       'Mar 2023', 'Mar 2024', 'Stock', 'Max', 'Diff%'],
      dtype='object')

In [5]:
cols = ['Type', 'Mar 2018', 'Mar 2019', 'Mar 2020', 'Mar 2021', 'Mar 2022', 'Mar 2023', 'Mar 2024', 'Stock']
df_latest_analysis = df_latest_analysis[cols]
df_latest_analysis.sort_values(by = 'Stock', ascending=True)

,Type,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Mar 2024,Stock
0,Sales,2710.0,3017.0,2987.0,2605.0,3336.0,3959.0,4189.0,3MINDIA
1,Profit,508.0,551.0,433.0,220.0,368.0,607.0,781.0,3MINDIA
2,Sales,20.0,61.0,108.0,195.0,298.0,338.0,395.0,5PAISA
3,Profit,-33.0,-25.0,-10.0,20.0,18.0,58.0,72.0,5PAISA
4,Sales,3806.0,4168.0,4186.0,4506.0,6086.0,6619.0,6372.0,AARTIIND
...,...,...,...,...,...,...,...,...,...
418,Sales,1710.0,1979.0,2493.0,2245.0,3378.0,4921.0,4448.0,WSTCSTPAPR
374,Sales,12529.0,19811.0,19258.0,12611.0,12528.0,14800.0,19527.0,YESBANK
375,Profit,6205.0,2349.0,-20838.0,-4762.0,1434.0,981.0,1538.0,YESBANK
377,Profit,2319.0,2434.0,956.0,1256.0,1400.0,264.0,323.0,ZEEL


In [6]:
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()

stock_df_final = pd.DataFrame()

for stock_name in stock_ids:

  stock_symbol = stock_name + '.NS'
  short_window = 20
  mid_window = 50
  long_window = 200
  moving_avg = 'SMA'

  # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
  stock_df = yfin.Ticker(stock_symbol).history(period='max', interval='1mo')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
  stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
  stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
  stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
  stock_df.dropna(axis = 0, inplace = True) # remove any null rows

  stock_df['Stock'] = stock_name

  stock_df['Close'] = round(stock_df['Close'],2)
  stock_df['Mon'] = stock_df.index.month_name().str.slice(stop=3)
  stock_df['Year'] = stock_df.index.strftime('%Y')
  stock_df['Year'] = stock_df['Mon'] + ' ' + stock_df['Year']
  cols = [ 'Year', 'Close']
  stock_df = stock_df[cols]
  stock_df = stock_df[(stock_df['Year'].str.contains("Mar")) ] #| (stock_df['Year'].str.contains(mon+' 2023'))]
  stock_df = stock_df.set_index('Year')
  stock_df = stock_df.T
  stock_df['Stock'] = stock_name
  stock_df.index.names = ['Type']
  stock_df = stock_df.reset_index()
  stock_df_final = pd.concat([stock_df_final, stock_df])

stock_df_final

yfinance: pandas_datareader support is deprecated & semi-broken so will be removed in a future verison. Just use yfinance.


Year,Type,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Mar 2024,Stock
0,Close,18667.90,23329.87,18105.45,29268.68,18941.44,22882.53,31193.35,3MINDIA
0,Close,203.15,144.82,113.15,249.70,344.70,283.00,505.05,5PAISA
0,Close,272.25,375.83,378.50,651.85,950.19,516.46,665.55,AARTIIND
0,Close,NaN,1155.80,1188.85,2420.85,2569.45,1610.75,1315.65,AAVAS
0,Close,5280.31,7078.48,14997.24,14666.08,17483.04,21929.65,27120.80,ABBOTINDIA
...,...,...,...,...,...,...,...,...,...
0,Close,308.00,470.79,236.75,349.14,735.10,570.60,525.50,VIPIND
0,Close,NaN,NaN,321.81,458.00,477.58,680.83,804.45,WESTLIFE
0,Close,1485.14,1493.49,1798.81,2207.28,1562.80,1310.90,1221.45,WHIRLPOOL
0,Close,205.94,249.56,193.43,408.18,584.32,364.47,480.10,WIPRO


In [7]:
#stock_df_final.drop(['Dec 2023',], axis=1, inplace=True)
#stock_df_final.rename(columns={'Mar 2024':'Dec 2023'}, inplace=True)

In [8]:
#df_gap_analysis = df_latest_analysis.append(stock_df_final)
df_gap_analysis = pd.concat([df_latest_analysis, stock_df_final], ignore_index = True)
df_gap_analysis['Growth%'] = (df_gap_analysis['Mar 2024'] - df_gap_analysis['Mar 2018'])*100/df_gap_analysis['Mar 2018']
df_gap_analysis['Growth%'] = round(df_gap_analysis['Growth%'],2)
df_gap_analysis

,Type,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Mar 2024,Stock,Growth%
0,Sales,2710.00,3017.00,2987.00,2605.00,3336.00,3959.00,4189.00,3MINDIA,54.58
1,Profit,508.00,551.00,433.00,220.00,368.00,607.00,781.00,3MINDIA,53.74
2,Sales,20.00,61.00,108.00,195.00,298.00,338.00,395.00,5PAISA,1875.00
3,Profit,-33.00,-25.00,-10.00,20.00,18.00,58.00,72.00,5PAISA,-318.18
4,Sales,3806.00,4168.00,4186.00,4506.00,6086.00,6619.00,6372.00,AARTIIND,67.42
...,...,...,...,...,...,...,...,...,...,...
585,Close,308.00,470.79,236.75,349.14,735.10,570.60,525.50,VIPIND,70.62
586,Close,NaN,NaN,321.81,458.00,477.58,680.83,804.45,WESTLIFE,NaN
587,Close,1485.14,1493.49,1798.81,2207.28,1562.80,1310.90,1221.45,WHIRLPOOL,-17.76
588,Close,205.94,249.56,193.43,408.18,584.32,364.47,480.10,WIPRO,133.13


In [9]:
df_gap_analysis[df_gap_analysis['Stock']  == 'ABBOTINDIA']

,Type,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Mar 2024,Stock,Growth%
420,Sales,3298.00,3679.00,4093.00,4310.00,4913.00,5349.00,5849.0,ABBOTINDIA,77.35
421,Profit,621.00,699.00,803.00,926.00,1080.00,1274.00,1618.0,ABBOTINDIA,160.55
460,Close,5280.31,7078.48,14997.24,14666.08,17483.04,21929.65,27120.8,ABBOTINDIA,413.62


In [10]:
stock_df_gap = pd.DataFrame()

stock_ids = df_gap_analysis['Stock'].unique()
#stock_ids = ['WSTCSTPAPR']
for stock_name in stock_ids:

  stock_symbol = stock_name
  #print(stock_symbol)
  tmp_df = df_gap_analysis[df_gap_analysis['Stock'] == stock_symbol]
  #print(tmp_df)
  try:
    gap_perc = round(tmp_df[tmp_df['Type'] == 'Profit']['Growth%'].values[0]  - tmp_df[tmp_df['Type'] == 'Close']['Growth%'].values[0],2)
    #print(gap_perc)
  except:
    pass
  tmp_df = pd.DataFrame()
  tmp_df['Stock'] = [stock_symbol]
  tmp_df['Gap%'] = [gap_perc]
  #print(tmp_df)
  stock_df_gap = pd.concat([stock_df_gap, tmp_df])

cols = ['Symbol', 'Criteria']
tmp_df = mypps[cols]
tmp_df.columns = ['Stock', 'Criteria']
stock_df_gap = pd.merge(stock_df_gap, tmp_df,  on='Stock')
stock_df_gap.sort_values(['Gap%'], ascending=[False]).head(30)

,Stock,Gap%,Criteria
10,AXISBANK,6009.28,V40
91,TANLA,2460.05,AR
92,TATACOMM,1963.32,AR
55,J&KBANK,1749.77,VR
126,BIRLAMONEY,499.50,AR
124,WSTCSTPAPR,199.66,VR
71,MOTILALOFS,186.95,V40N
46,ICICIBANK,157.96,V40
49,INDUSINDBK,128.88,VR
60,KANSAINER,117.70,V40N


In [11]:
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    #start = datetime.datetime(*map(int, start_date.split('-')))
    #end = datetime.datetime(*map(int, end_date.split('-')))
    #print(stock_symbol)
    stock_df = yfin.Ticker(stock_symbol+'.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    #stock_df.columns = ['Close']

    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Stock'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Max'] = round(max(stock_df['Close']),2)
    #stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
    #stock_df['Min'] = round(min(stock_df['Close']),2)
    #stock_df['Gained%'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
    #stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    #stock_df['Close'] = round(stock_df['Close'],0)
    #stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['ATH%'] = round((stock_df['Max'] - stock_df['Close'])*100/stock_df['Close'],2)
    #stock_df['Sector'] = stock_symbol
    stock_df = stock_df.tail(1)
    cols = ['Stock', 'ATH%']
    return stock_df[cols]

In [12]:
stock_df_max = pd.DataFrame()
for stock_name in stock_ids:
  #print(stock_name)
  tmp_df = stock_prec_dev(stock_name)
  stock_df_max = pd.concat([stock_df_max, tmp_df])
stock_df_max

,Stock,ATH%
Date,,
2024-06-14 00:00:00+05:30,3MINDIA,2.25
2024-06-14 00:00:00+05:30,5PAISA,37.97
2024-06-14 00:00:00+05:30,AARTIIND,69.64
2024-06-14 00:00:00+05:30,AAVAS,72.39
2024-06-14 00:00:00+05:30,ABSLAMC,0.00
...,...,...
2024-06-14 00:00:00+05:30,SPORTKING,26.43
2024-06-14 00:00:00+05:30,TASTYBITE,83.76
2024-06-14 00:00:00+05:30,TIPSINDLTD,12.46


In [16]:
stock_df_final = pd.merge(stock_df_gap, stock_df_max,  on='Stock')
stock_df_final['InPF'] = 0
stock_df_final.loc[stock_df_final['Stock'].isin(mypfs), 'InPF'] = 1
stock_df_final[(stock_df_final['Gap%'] >=50) & (stock_df_final['Gap%'] <= 299)].sort_values(by = 'ATH%', ascending=False)

,Stock,Gap%,Criteria,ATH%,InPF
71,MOTILALOFS,186.95,V40N,294.58,0
66,LUXIND,77.82,V40N,209.68,0
109,CUB,67.45,AR,58.20,0
60,KANSAINER,117.70,V40N,55.37,0
50,INFIBEAM,69.90,AR,54.10,0
87,SIS,71.85,V40N,52.99,0
49,INDUSINDBK,128.88,VR,30.79,0
61,KOTAKBANK,89.66,V40,28.57,0
89,SUNTV,67.25,V40N,17.88,0
124,WSTCSTPAPR,199.66,VR,15.52,0


In [14]:
df_gap_analysis[df_gap_analysis['Stock']  == 'TATACOMM']

,Type,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Mar 2024,Stock,Growth%
322,Sales,16772.00,16525.00,17068.00,17100.00,16725.00,17838.00,20969.0,TATACOMM,25.02
323,Profit,46.00,343.00,140.00,1609.00,2000.00,2063.00,1163.0,TATACOMM,2428.26
573,Close,355.86,356.07,221.64,1014.82,1185.89,1228.74,2010.4,TATACOMM,464.94


In [15]:
stock_df_final[stock_df_final['Gap%']  == 199.66]['Stock'].unique()

array(['WSTCSTPAPR'], dtype=object)